# Feature Engineering Tests

### - Pagerank

###Loading all packages and datasets

In [0]:
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql import functions as f
from pyspark.sql import SQLContext
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import isnan, when, count, col, isnull, percent_rank
from pyspark.sql.functions import min
from pyspark.sql.functions import col, max
from pyspark.sql.functions import format_string
from pyspark.sql.functions import substring
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import concat
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import lit
from pyspark.sql.functions import to_utc_timestamp
from pyspark.sql.functions import expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import instr
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.linalg import Vectors
from pyspark.mllib.evaluation import MulticlassMetrics


from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data_BASE_DIR = "dbfs:/mnt/mids-w261/"
display(dbutils.fs.ls(f"{data_BASE_DIR}"))

path,name,size,modificationTime
dbfs:/mnt/mids-w261/HW5/,HW5/,0,0
dbfs:/mnt/mids-w261/datasets_final_project/,datasets_final_project/,0,0
dbfs:/mnt/mids-w261/datasets_final_project_2022/,datasets_final_project_2022/,0,1656617686000


In [0]:
# Inspect the Mount's Final Project folder 
# Please IGNORE dbutils.fs.cp("/mnt/mids-w261/datasets_final_project/stations_data/", "/mnt/mids-w261/datasets_final_project_2022/stations_data/", recurse=True)
data_BASE_DIR = "dbfs:/mnt/mids-w261/datasets_final_project_2022/"
display(dbutils.fs.ls(f"{data_BASE_DIR}"))

path,name,size,modificationTime
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/,parquet_airlines_data/,0,1656618287000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_1y/,parquet_airlines_data_1y/,0,1656630272000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0,1656630114000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0,1656630205000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data/,parquet_weather_data/,0,1656622074000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_1y/,parquet_weather_data_1y/,0,1656631614000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_3m/,parquet_weather_data_3m/,0,1656630651000
dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_6m/,parquet_weather_data_6m/,0,1656631047000
dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/,stations_data/,0,1656713663000


### Init Script and SAS Token

In [0]:
#Initializes blob storage credentials/location
blob_container = "w261-sec4-group2" # The name of your container created in https://portal.azure.com
storage_account = "kdevery" # The name of your Storage account created in https://portal.azure.com
secret_scope = "sec4-group2" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261-key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

#Points to SAS token
spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

### Pagerank

In [0]:
# Read merged data data

df = spark.read.parquet(f"{blob_url}/merged_cleaned_data")

In [0]:
display(df)

DEP_DEL15,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,two_hrs_pre_flight_utc,Date_Time_sched_dep_utc,Date_Time_sched_arrival_utc,OP_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyRelativeHumidity,HourlySkyConditions,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,HourlyWindGustSpeed
0.0,2015,1,1,14,3,2015-01-14,2015-01-14T19:00:00.000+0000,2015-01-14T21:00:00.000+0000,2015-01-14T23:00:00.000+0000,EV,N937EV,ABE,ATL,1601,1820,-9,139.0,692.0,3,118.9,30.29,13,25,0.0,null,60,OVC:08 32,30.31,29.87,10.0,21,60,3,null
0.0,2015,1,1,22,4,2015-01-22,2015-01-22T19:00:00.000+0000,2015-01-22T21:00:00.000+0000,2015-01-22T23:00:00.000+0000,EV,N854AS,ABE,ATL,1601,1820,-3,139.0,692.0,3,118.9,30.22,29,37,0.0,null,73,SCT:04 22 SCT:04 27 BKN:07 37,30.23,29.8,7.0,34,280,6,null
0.0,2015,1,3,17,2,2015-03-17,2015-03-17T18:00:00.000+0000,2015-03-17T20:00:00.000+0000,2015-03-17T23:00:00.000+0000,EV,N14116,ABE,ORD,1648,1805,7,137.0,654.0,3,118.9,29.67,33,53,0.0,null,47,SCT:04 47 BKN:07 60,29.67,29.26,10.0,44,300,20,40
0.0,2015,1,3,17,2,2015-03-17,2015-03-17T18:00:00.000+0000,2015-03-17T20:00:00.000+0000,2015-03-17T22:00:00.000+0000,EV,N922EV,ABE,ATL,1600,1808,-8,128.0,692.0,3,118.9,29.67,33,53,0.0,null,47,SCT:04 47 BKN:07 60,29.67,29.26,10.0,44,300,20,40
0.0,2015,1,3,30,1,2015-03-30,2015-03-30T08:00:00.000+0000,2015-03-30T10:00:00.000+0000,2015-03-30T12:00:00.000+0000,DL,N968AT,ABE,ATL,640,848,-5,128.0,692.0,3,118.9,29.94,29,34,null,null,82,OVC:08 33,29.95,29.53,9.0,32,0,0,null
0.0,2015,1,3,30,1,2015-03-30,2015-03-30T08:00:00.000+0000,2015-03-30T10:00:00.000+0000,2015-03-30T11:00:00.000+0000,EV,N882AS,ABE,DTW,600,746,-6,106.0,425.0,2,118.9,29.94,29,34,null,null,82,OVC:08 33,29.95,29.53,9.0,32,0,0,null
0.0,2015,1,3,30,1,2015-03-30,2015-03-30T08:00:00.000+0000,2015-03-30T10:00:00.000+0000,2015-03-30T12:00:00.000+0000,EV,N14105,ABE,ORD,600,720,-16,140.0,654.0,3,118.9,29.94,29,34,null,null,82,OVC:08 33,29.95,29.53,9.0,32,0,0,null
1.0,2015,2,4,8,3,2015-04-08,2015-04-08T19:00:00.000+0000,2015-04-08T21:00:00.000+0000,2015-04-08T23:00:00.000+0000,EV,N16561,ABE,ORD,1740,1850,108,130.0,654.0,3,118.9,30.25,29,46,0.0,null,51,OVC:08 31,30.26,29.83,10.0,39,110,9,null
1.0,2015,2,4,8,3,2015-04-08,2015-04-08T19:00:00.000+0000,2015-04-08T21:00:00.000+0000,2015-04-08T23:00:00.000+0000,EV,N840AS,ABE,DTW,1724,1905,259,101.0,425.0,2,118.9,30.25,29,46,0.0,null,51,OVC:08 31,30.26,29.83,10.0,39,110,9,null
0.0,2015,2,5,8,5,2015-05-08,2015-05-08T15:00:00.000+0000,2015-05-08T17:00:00.000+0000,2015-05-08T18:00:00.000+0000,EV,N981EV,ABE,DTW,1300,1443,-4,103.0,425.0,2,118.9,30.14,61,79,0.0,null,54,CLR:00,30.14,29.72,10.0,68,null,7,null


In [0]:
df.count()

Out[7]: 41553980

In [0]:
df = df.dropDuplicates()
display(df)

DEP_DEL15,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,two_hrs_pre_flight_utc,Date_Time_sched_dep_utc,Date_Time_sched_arrival_utc,OP_CARRIER,TAIL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyRelativeHumidity,HourlySkyConditions,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,HourlyWindGustSpeed
1.0,2019,2,5,3,5,2019-05-03,2019-05-03T14:00:00.000+0000,2019-05-03T16:00:00.000+0000,2019-05-03T18:00:00.000+0000,OO,N463SW,ABE,DTW,1230,1421,57,111.0,425.0,2,118.9,30.1,49,54,null,null,83,OVC:08 6,30.1,29.68,9.0,51,110,6,null
0.0,2021,2,5,7,5,2021-05-07 00:00:00,2021-05-07T16:00:00.000+0000,2021-05-07T18:00:00.000+0000,2021-05-07T21:00:00.000+0000,G4,223NV,ABE,SFB,1452,1712,-3,140.0,882.0,4,117.4,29.87,32,63,0.0,null,31,CLR:00,29.87,29.46,10.0,48,100,8,null
0.0,2016,1,3,20,7,2016-03-20,2016-03-20T09:00:00.000+0000,2016-03-20T11:00:00.000+0000,2016-03-20T15:00:00.000+0000,WN,N777QC,ABQ,BWI,530,1105,-11,215.0,1670.0,7,1618.5,30.3,19,38,0.0,null,46,FEW:02 150,30.21,24.92,10.0,31,70,13,null
0.0,2021,3,7,10,6,2021-07-10 00:00:00,2021-07-10T18:00:00.000+0000,2021-07-10T20:00:00.000+0000,2021-07-10T22:00:00.000+0000,DL,N390DN,ABQ,MSP,1407,1744,-13,157.0,981.0,4,1618.4,30.16,47,91,0.0,null,22,FEW:02 90,29.86,24.81,10.0,64,null,5,null
0.0,2019,2,5,7,2,2019-05-07,2019-05-07T18:00:00.000+0000,2019-05-07T20:00:00.000+0000,2019-05-07T21:00:00.000+0000,OO,N675BR,ABY,ATL,1614,1715,-2,61.0,145.0,1,57.9,30.1,54,88,0.0,null,31,CLR:00,30.09,29.89,10.0,67,130,7,null
0.0,2020,4,11,30,1,2020-11-30 00:00:00,2020-11-30T10:00:00.000+0000,2020-11-30T12:00:00.000+0000,2020-11-30T13:00:00.000+0000,MQ,N822AE,ACT,DFW,646,740,-6,54.0,89.0,1,152.4,30.41,24,39,0.0,null,55,CLR:00,30.4,29.86,10.0,33,350,15,28
0.0,2016,3,7,18,1,2016-07-18,2016-07-18T09:00:00.000+0000,2016-07-18T11:00:00.000+0000,2016-07-18T14:00:00.000+0000,NK,N633NK,ACY,MCO,745,1002,-15,137.0,852.0,4,18.3,30.14,72,77,0.0,null,85,CLR:00,30.13,30.07,10.0,74,210,7,null
0.0,2017,3,9,23,6,2017-09-23,2017-09-24T05:00:00.000+0000,2017-09-24T07:00:00.000+0000,2017-09-23T11:00:00.000+0000,AS,N767AS,ANC,SEA,2355,410,2,195.0,1448.0,6,36.6,29.45,49,50,0.0,null,96,SCT:04 6 BKN:07 110,29.45,29.31,10.0,49,330,3,null
0.0,2017,4,11,3,5,2017-11-03,2017-11-04T03:00:00.000+0000,2017-11-04T05:00:00.000+0000,2017-11-03T10:00:00.000+0000,DL,N699DL,ANC,MSP,2145,558,-20,313.0,2519.0,11,36.6,30.49,31,32,0.0,FZ:8 FG:2 |FG |,96,VV:09 2,30.49,30.35,0.25,32,40,3,null
0.0,2019,2,6,8,6,2019-06-08,2019-06-08T19:00:00.000+0000,2019-06-08T21:00:00.000+0000,2019-06-09T01:00:00.000+0000,AS,N435AS,ANC,SEA,1350,1820,9,210.0,1448.0,6,36.6,30.1,49,56,0.0,null,77,FEW:02 30 SCT:04 60 BKN:07 100,30.1,29.96,10.0,52,300,8,null


In [0]:
df.count()

Out[9]: 41553978

In [0]:
train_df = df.filter(df.YEAR < 2021)

In [0]:
train_df.count()

Out[11]: 35661519

In [0]:
# select origin and dest only

node_df = train_df.select('ORIGIN', 'DEST')
display(node_df)
node_df.count()

ORIGIN,DEST
ABE,DTW
ABQ,BWI
ABY,ATL
ACT,DFW
ACY,MCO
ANC,SEA
ANC,MSP
ANC,SEA
ANC,PDX
APN,DTW


Out[12]: 35661519

In [0]:
# convert to RDD

airport_rdd = node_df.rdd
airport_rdd.take(5)

Out[13]: [Row(ORIGIN='ABE', DEST='DTW'),
 Row(ORIGIN='ABQ', DEST='PHX'),
 Row(ORIGIN='ABY', DEST='ATL'),
 Row(ORIGIN='AEX', DEST='ATL'),
 Row(ORIGIN='AGS', DEST='ATL')]

In [0]:
# convert to same format

def format(line):
  origin = line[0]
  dest = line[1]
  yield ((origin, dest), 1)
         
airRDD = airport_rdd.flatMap(format).reduceByKey(lambda x,y: x+y)

In [0]:
# cache rdd, reduced down to 8298 rows
airRDD.cache()

Out[15]: PythonRDD[74] at RDD at PythonRDD.scala:58

In [0]:
def matchformat(line):
  node = line[0][0]
  edge = line [0][1]
  count = line[1]
  edge_dict = {edge:count}
  yield (node, str(edge_dict))

airRDD1 = airRDD.flatMap(matchformat)

In [0]:
airRDD1.take(5)

Out[17]: [('LIT', "{'DEN': 4090}"),
 ('SMF', "{'SLC': 10332}"),
 ('AUS', "{'IAH': 14394}"),
 ('BWI', "{'MIA': 5294}"),
 ('DCA', "{'SFO': 3902}")]

In [0]:
# from HW5 wikiRDD Format

# 'node' \t {'edge' : number}

# ["2\t{'3': 1}",
#  "3\t{'2': 2}",
#  "4\t{'1': 1, '2': 1}",
#  "5\t{'4': 3, '2': 1, '6': 1}",
#  "6\t{'2': 1, '5': 2}",
#  "7\t{'2': 1, '5': 1}",
#  "8\t{'2': 1, '5': 1}",
#  "9\t{'2': 1, '5': 1}",
#  "10\t{'5': 1}",
#  "11\t{'5': 2}"]

In [0]:
# initialize graph

def initGraph(dataRDD):
    """
    Spark job to read in the raw data and initialize an 
    adjacency list representation with a record for each
    node (including dangling nodes).
    
    Returns: 
        graphRDD -  a pair RDD of (node_id , (score, edges))
        
    NOTE: The score should be a float, but you may want to be 
    strategic about how format the edges... there are a few 
    options that can work. Make sure that whatever you choose
    is sufficient for Question 8 where you'll run PageRank.
    """
    ############## YOUR CODE HERE ###############

    # write any helper functions here
    
    def parse(line):
        node = line[0]
        edges = line[1]
        return (node, ast.literal_eval(edges))

    def getEdgeNodes(line):
        yield line[0]
    
    def getAllNodes(line):
        yield line[0]
        part2 = set(line[1].keys())
        for p in part2:
            yield p
            
    def dict_to_string(d):
        '''from stackoverflow'''
        edge_dict = d
        res = []
        for key, val in edge_dict.items():
            res += [key] * val
        return res
    
    def emitEntries(line):
        node = line[0]
        edges = dict_to_string(line[1])
        yield (node, (1.0/N.value, edges))

    # write your main Spark code here
    
    # parse and cache dataRDD
    parsed_rdd = dataRDD.map(parse).cache()
    
    # get all nodes
    all_nodes = parsed_rdd.flatMap(getAllNodes).distinct().cache() 
    
    #broadcast count of all nodes
    N = sc.broadcast(all_nodes.count())
    
    # get list of edge nodes
    nodes_with_edges = parsed_rdd.flatMap(getEdgeNodes).cache()
    
    # subtract to get dangling nodes, map proper format
    nodes_wo_edgesRDD = all_nodes.subtract(nodes_with_edges) \
                            .map(lambda x: (x, (1.0/N.value, [])))
    
    # map proper format on all edge nodes
    nodes_w_edgesRDD = parsed_rdd.flatMap(emitEntries)

    # combine edges and non edges rdds
    graphRDD = nodes_w_edgesRDD \
                    .union(nodes_wo_edgesRDD) \
                    .reduceByKey(lambda x,y : x+y)

    ############## (END) YOUR CODE ##############
    
    return graphRDD

In [0]:
import re
import ast
import time

start = time.time()
flightGraph = initGraph(airRDD1)
print(f'... test graph initialized in {time.time() - start} seconds.')

... test graph initialized in 91.53105759620667 seconds.


In [0]:
# custom accumulator
from pyspark.accumulators import AccumulatorParam

class FloatAccumulatorParam(AccumulatorParam):
    """
    Custom accumulator for use in page rank to keep track of various masses.
    
    IMPORTANT: accumulators should only be called inside actions to avoid duplication.
    We stringly recommend you use the 'foreach' action in your implementation below.
    """
    def zero(self, value):
        return value
    def addInPlace(self, val1, val2):
        return val1 + val2

In [0]:
# pagerank calculation
def runPageRank(graphInitRDD, alpha = 0.15, maxIter = 10, verbose = True):
    """
    Spark job to implement page rank
    Args: 
        graphInitRDD  - pair RDD of (node_id , (score, edges))
        alpha         - (float) teleportation factor
        maxIter       - (int) stopping criteria (number of iterations)
        verbose       - (bool) option to print logging info after each iteration
    Returns:
        steadyStateRDD - pair RDD of (node_id, pageRank)
    """
    # teleportation:
    a = sc.broadcast(alpha)
    
    # damping factor:
    d = sc.broadcast(1-a.value)
    
    # initialize accumulators for dangling mass & total mass
    mmAccum = sc.accumulator(0.0, FloatAccumulatorParam())
    totAccum = sc.accumulator(0.0, FloatAccumulatorParam())
    
    ############## YOUR CODE HERE ###############  
    # write your helper functions here, 
    # please document the purpose of each clearly 
    # for reference, the master solution has 5 helper functions.
    
    def getNoInNodes(line):
        '''produce an rdd of all nodes with their graph structure and zero mass'''
        node = line[0]
        edges = line[1][1]
        yield(node, (edges, 0.0))
            
    def parseNeighbors(line):
        """Parses payload to get neighbor list in correct format"""
        node = line[0]
        rank = line[1][0]
        neighbors = line[1][1]
        yield(node, (neighbors, rank))

    def computeContribs(line):
        """Calculates neighbor contributions to the rank of others."""
        node = line[0]
        neighbors = line[1][0]
        rank = line[1][1]
        num_neigh = len(neighbors)
        if neighbors != []:
            for n in neighbors:
                yield (n, ([], rank / num_neigh))
                #totAccum.add(rank / num_neigh)
            
    def computeDanglingMass(line, mmAccum):
        """Adds dangling mass to accumulator"""
        neighbors = line[1][0]
        rank = line[1][1]
        if neighbors == []:
            #totAccum.add(rank)
            mmAccum.add(rank)        

#     write your main Spark Job here (including the for loop to iterate)
#     for reference, the master solution is 21 lines including comments & whitespace

    N = sc.broadcast(graphInitRDD.count())
    
    # rdd of edges for mass to be distributed
    currentRDD = graphInitRDD.flatMap(parseNeighbors).cache()
        
    # all nodes RDD
    allNodesRDD = graphInitRDD.flatMap(getNoInNodes).cache() 
    
    for i in range(maxIter):
        #for each entry, check if there are no edges and add to dangling mass, broadcast value
        danglingmass = currentRDD.foreach(lambda x: computeDanglingMass(x, mmAccum))
        mmAccum_bc = sc.broadcast(mmAccum.value)

        #distributed mass RDD  
        distRDD = currentRDD.flatMap(computeContribs) \
                            .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])).cache()
        # join all nodes with structure to distributed RDD, reduce, and distribute mass with teleport 
        stateRDD = allNodesRDD.union(distRDD) \
                                    .reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1])) \
                                    .mapValues(lambda x: (x[0], (((x[1] + mmAccum_bc.value/N.value) * d.value) + (a.value * (1/N.value)))))

        if verbose:
            print('Iteration #:', i)
            print("Dangling Mass:", mmAccum_bc.value)
#             print("Total Mass:", totAccum.value)             

        currentRDD = stateRDD.cache()
        mmAccum.value = 0.0

    steadyStateRDD = stateRDD.map(lambda x: ((x[0]), x[1][1])).cache() 
    ############## (END) YOUR CODE ###############
    
    return steadyStateRDD

In [0]:
nIter = 30
start = time.time()
full_results = runPageRank(flightGraph, alpha = 0.15, maxIter = nIter, verbose = True)
print(f'\n...trained {nIter} iterations in {time.time() - start} seconds.\n')

Iteration #: 0
Dangling Mass: 0.0026246719160104987
Iteration #: 1
Dangling Mass: 0.0003995563546682649
Iteration #: 2
Dangling Mass: 0.0003945921860930919
Iteration #: 3
Dangling Mass: 0.00039458111117632316
Iteration #: 4
Dangling Mass: 0.0003945810864685036
Iteration #: 5
Dangling Mass: 0.0003945810864133812
Iteration #: 6
Dangling Mass: 0.0003945810864132582
Iteration #: 7
Dangling Mass: 0.0003945810864132579
Iteration #: 8
Dangling Mass: 0.0003945810864132579
Iteration #: 9
Dangling Mass: 0.0003945810864132579
Iteration #: 10
Dangling Mass: 0.0003945810864132579
Iteration #: 11
Dangling Mass: 0.0003945810864132579
Iteration #: 12
Dangling Mass: 0.0003945810864132579
Iteration #: 13
Dangling Mass: 0.0003945810864132579
Iteration #: 14
Dangling Mass: 0.0003945810864132579
Iteration #: 15
Dangling Mass: 0.0003945810864132579
Iteration #: 16
Dangling Mass: 0.0003945810864132579
Iteration #: 17
Dangling Mass: 0.0003945810864132579
Iteration #: 18
Dangling Mass: 0.0003945810864132579
It

In [0]:
print(f'Top 40 ranked nodes:\n')
top_40 = full_results.takeOrdered(40, key=lambda x: -x[1])
# print results from 20th to 40th highest PageRank
for result in top_40:
    print(result)

Top 40 ranked nodes:

('DFW', 0.06467809637495084)
('CVG', 0.057523602452784275)
('SFB', 0.05685779524258357)
('BNA', 0.049287935252991924)
('ATL', 0.03920216771605087)
('TLH', 0.03409351008635021)
('IAH', 0.033386574541086)
('DEN', 0.032631533097708526)
('PHX', 0.03256628707761307)
('MSY', 0.02993436162506286)
('JAC', 0.029079979255968284)
('PHL', 0.028797407367733317)
('SMF', 0.028300134047295724)
('ROW', 0.02824293808249145)
('SEA', 0.026493067833880033)
('DTW', 0.02591797872959223)
('BOS', 0.02366512481384447)
('BMI', 0.022659704611319706)
('HDN', 0.02064577526323217)
('ORD', 0.012019927701236466)
('MYR', 0.0119468629679941)
('RSW', 0.011567392314298692)
('IND', 0.010226864553567739)
('PDX', 0.008649508047810067)
('MCO', 0.008078539721162445)
('MEM', 0.007261339849401185)
('SFO', 0.00673377778508491)
('EUG', 0.006118292203734419)
('RDU', 0.005056556622385537)
('SLC', 0.004888859660660109)
('LAS', 0.0047454787583850406)
('SYR', 0.004692654215440953)
('BUR', 0.0044282380310400864)
('

In [0]:
pagerank_df = full_results.toDF()
pagerank_df = pagerank_df.withColumnRenamed("_1","ICAO")
pagerank_df = pagerank_df.withColumnRenamed("_2","Pagerank_Score")

display(pagerank_df)

ICAO,Pagerank_Score
GUC,3.945810864132579E-4
USA,0.014347930826777938
PHX,0.0040574727437268565
CSG,3.945810864132579E-4
SMF,3.945810864132579E-4
AVP,3.945810864132579E-4
MOT,3.945810864132579E-4
GRK,7.299750098645272E-4
RIW,3.945810864132579E-4
LRD,3.945810864132579E-4


In [0]:
# write dataframe to blob
pagerank_df.write.mode("overwrite").parquet(f"{blob_url}/pagerank_scores")

In [0]:
display(dbutils.fs.ls(f"{blob_url}"))

path,name,size,modificationTime
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data/,feature_engineered_data/,0,1668652882000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_data_test/,feature_engineered_data_test/,0,1668652922000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/feature_engineered_train_data/,feature_engineered_train_data/,0,1668559613000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data/,merged_cleaned_data/,0,1668557804000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_test/,merged_cleaned_data_test/,0,1668574917000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_cleaned_data_train/,merged_cleaned_data_train/,0,1668574895000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/merged_data/,merged_data/,0,1667926411000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/pagerank_scores/,pagerank_scores/,0,1668653872000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/premerge_airline_data/,premerge_airline_data/,0,1667925382000
wasbs://w261-sec4-group2@kdevery.blob.core.windows.net/premerge_weather_station_data/,premerge_weather_station_data/,0,1667925572000
